# `Time Series Plotting`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# `Purpose & Outcome:`

+ 

In [4]:
import pandas as pd
import os,datetime


In [ ]:
# SalesDataNoCancels.csv
# 

In [2]:
# Find a file outside your directory:

def os_any_dir_search(file):
    u=[]
    for p,n,f in os.walk(os.getcwd()):
        
        for a in f:
            a = str(a)
            if a.endswith(file): # can be (.csv) or a file like I did and search 
#                 print(a)
#                 print(p)
                t=pd.read_csv(p+'/'+file)
                u.append(p+'/'+a)
    return t,u

In [28]:
retail_sales_cleaned = os_any_dir_search('SalesDataNoCancels.csv')[0]
# fake_users=fake_users.iloc[:,1:]
retail_sales_cleaned.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CanceledQty,Matches
0,563614,23345,DOLLY GIRL BEAKER,200,8/18/2011 8:51,1.08,12415,Australia,1,False
1,568708,23391,I LOVE LONDON MINI BACKPACK,4,9/28/2011 15:41,4.15,12393,Australia,1,False
2,556917,22418,10 COLOUR SPACEBOY PEN,48,6/15/2011 13:37,0.85,12415,Australia,1,False
3,543989,20973,12 PENCIL SMALL TUBE WOODLAND,384,2/15/2011 9:52,0.55,12415,Australia,1,False
4,547659,20984,12 PENCILS TALL TUBE POSY,12,3/24/2011 13:05,0.85,12434,Australia,1,False


In [30]:
retail_sales_cleaned['InvoiceDate']=pd.to_datetime(retail_sales_cleaned['InvoiceDate'],
                    infer_datetime_format=True).head()

retail_sales_cleaned['Date']=retail_sales_cleaned['InvoiceDate'].dt.strftime('%m/%d/%Y')
retail_sales_cleaned['Time']=retail_sales_cleaned['InvoiceDate'].dt.strftime('%H:%M')
retail_sales_cleaned.head()
# retail_sales_cleaned['InvoiceDate'].dt.strftime('%H:%M')
# format='%m/%d/%Y %H:%M'

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CanceledQty,Matches,Date,Time
0,563614,23345,DOLLY GIRL BEAKER,200,2011-08-18 08:51:00,1.08,12415,Australia,1,False,08/18/2011,08:51
1,568708,23391,I LOVE LONDON MINI BACKPACK,4,2011-09-28 15:41:00,4.15,12393,Australia,1,False,09/28/2011,15:41
2,556917,22418,10 COLOUR SPACEBOY PEN,48,2011-06-15 13:37:00,0.85,12415,Australia,1,False,06/15/2011,13:37
3,543989,20973,12 PENCIL SMALL TUBE WOODLAND,384,2011-02-15 09:52:00,0.55,12415,Australia,1,False,02/15/2011,09:52
4,547659,20984,12 PENCILS TALL TUBE POSY,12,2011-03-24 13:05:00,0.85,12434,Australia,1,False,03/24/2011,13:05


In [34]:
retail_sales_cleaned['Day_of_Week']=retail_sales_cleaned['InvoiceDate'].dt.strftime('%A')
retail_sales_cleaned['Month']=retail_sales_cleaned['InvoiceDate'].dt.strftime('%B')
retail_sales_cleaned.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CanceledQty,Matches,Date,Time,Day_of_Week,Month
0,563614,23345,DOLLY GIRL BEAKER,200,2011-08-18 08:51:00,1.08,12415,Australia,1,False,08/18/2011,08:51,Thursday,August
1,568708,23391,I LOVE LONDON MINI BACKPACK,4,2011-09-28 15:41:00,4.15,12393,Australia,1,False,09/28/2011,15:41,Wednesday,September
2,556917,22418,10 COLOUR SPACEBOY PEN,48,2011-06-15 13:37:00,0.85,12415,Australia,1,False,06/15/2011,13:37,Wednesday,June
3,543989,20973,12 PENCIL SMALL TUBE WOODLAND,384,2011-02-15 09:52:00,0.55,12415,Australia,1,False,02/15/2011,09:52,Tuesday,February
4,547659,20984,12 PENCILS TALL TUBE POSY,12,2011-03-24 13:05:00,0.85,12434,Australia,1,False,03/24/2011,13:05,Thursday,March


In [ ]:
# Bar Plot

In [ ]:
# Dot/line plot different colors
# monthly

In [ ]:
# daily

In [ ]:
# hourly

In [ ]:
# day month and dollar amounts

# `Citations & Help:`

# ◔̯◔

`Date Time Help`

https://www.datacamp.com/community/tutorials/converting-strings-datetime-objects

https://stackoverflow.com/questions/38067704/how-to-change-the-datetime-format-in-pandas

https://stackoverflow.com/questions/38709558/how-to-format-a-date-column-in-pandas

`Plotting Help:`

https://machinelearningmastery.com/time-series-data-visualization-with-python/

https://unidata.github.io/python-training/workshop/Time_Series/basic-time-series-plotting/